In [1]:
import openpyxl
import pandas as pd
import os

In [2]:
excelname='ma'
sheetname='10ma'
wb=openpyxl.load_workbook(str(excelname)+".xlsx")
ws=wb[str(sheetname)]

In [3]:
dataset=[]
i=1
while(ws.cell(row=i,column=1).value!=None):
    row=[]
    j=1
    while(ws.cell(row=i,column=j).value!=None):
        row.append(ws.cell(row=i,column=j).value)
        j+=1
    dataset.append(row)
    i+=1

In [4]:
trade=[[0 for i in range(11)]for j in range(len(dataset))]
trade[0][0]='date'
trade[0][1]='buy'
trade[0][2]='hold'
trade[0][3]='sell'
trade[0][4]='number'
trade[0][5]='value'
trade[0][6]='period_return'
trade[0][7]='closeprice'
trade[0][8]='10ma'
trade[0][9]='holding_period'
trade[0][10]='annualized_return'

In [5]:
for i in range(1,len(trade)):
    trade[i][0]=dataset[i][0]
    trade[i][7]=float(dataset[i][7])
#10ma
for i in range(10,len(trade)):
    temp=0
    for j in range(0,10):
        temp+=trade[i-j][7]
    trade[i][8]=temp/10

In [6]:
for i in range(10,len(trade)):
    if(trade[i][7]>trade[i][8])and(trade[i-1][2]==0):
        trade[i][1]=1
        trade[i][2]=1
        trade[i][3]=0
    elif(trade[i][7]<trade[i][8])and(trade[i-1][2]==1):
        trade[i][1]=0
        trade[i][2]=0
        trade[i][3]=1
    elif(trade[i-1][2]>0):
        trade[i][1]=0
        trade[i][2]=1
        trade[i][3]=0
initial=10000

In [7]:
for i in range(10,len(trade)):
    if(trade[i][1]==1)and(trade[i][2]==1)and(trade[i][3]==0):
        trade[i][4]=initial/trade[i][7]
        trade[i][5]=trade[i][4]*trade[i][7]
        trade[i][6]=round((trade[i][4]*trade[i][7]-initial)/initial,4)
    elif(trade[i][1]==0)and(trade[i][2]==0)and(trade[i][3]==1):
        trade[i][4]=0
        trade[i][5]=trade[i-1][4]*trade[i][7]
        trade[i][6]=round((trade[i-1][4]*trade[i][7]-initial)/initial,4)
    elif(trade[i][1]==0)and(trade[i][2]==1)and(trade[i][3]==0):
        trade[i][4]=trade[i-1][4]
        trade[i][5]=trade[i][4]*trade[i][7]
        trade[i][6]=round((trade[i][4]*trade[i][7]-initial)/initial,4)

In [9]:
import datetime
from datetime import date
for i in range(1,len(trade)):
    trade[i][0]=datetime.datetime.strptime(trade[i][0],'%Y-%m-%d')

In [10]:
#holding_period
# for i in range(10,len(trade)):
#     if(trade[i][1]==1)and(trade[i][2]==1)and(trade[i][3]==0):
#         first=trade[i][0]
#         j=0
#         while((trade[i+j][1]!=0)or(trade[i+j][2]!=0)or(trade[i+j][3]!=0)):
#             if(trade[i+j][1]==0)and(trade[i+j][2]==0)and(trade[i+j][3]==1):
#                 trade[i+j][9]=(trade[i+j][0]-first).days+1
#                 break
#             else:
#                 trade[i+j][9]=(trade[i+j][0]-first).days+1
#                 j+=1
#             break

In [12]:
# holding_period
start=0
end=0
for i in range(1,len(trade)):
    if(trade[i][1]==1):
        trade[i][9]=1
        start=trade[i][0]
    elif((trade[i][2]==1)or(trade[i][3]==1)and(trade[i][1]!=1)):
        end=trade[i][0]
        trade[i][9]=(end-start).days+1

In [14]:
for i in range(1,len(trade)):
    if(trade[i][9]!=0):
        trade[i][10]=(1+trade[i][6])**(365/trade[i][9])-1

In [15]:
annual_return=1
trading_counts=0
for i in range(1,len(trade)):
    if(trade[i][3]==1):
        annual_return*=(1+trade[i][6])
        trading_counts+=1
final_return=annual_return**(1/trading_counts)-1
print(annual_return)
print(trading_counts)
print(final_return)

0.9765753225396776
285
-8.316635349714385e-05


In [27]:
pd.DataFrame(trade).head(20)

,0,1,2,3,4,5,6,7,8,9,10
0,date,buy,hold,sell,number,value,period_return,closeprice,10ma,holding_period,annualized_return
1,2003-07-01 00:00:00,0,0,0,0,0,0,38.05,0,0,0
2,2003-07-02 00:00:00,0,0,0,0,0,0,38.69,0,0,0
3,2003-07-03 00:00:00,0,0,0,0,0,0,39,0,0,0
4,2003-07-04 00:00:00,0,0,0,0,0,0,39.26,0,0,0
5,2003-07-07 00:00:00,0,0,0,0,0,0,41,0,0,0
6,2003-07-08 00:00:00,0,0,0,0,0,0,41.19,0,0,0
7,2003-07-09 00:00:00,0,0,0,0,0,0,41.22,0,0,0
8,2003-07-10 00:00:00,0,0,0,0,0,0,40.05,0,0,0
9,2003-07-11 00:00:00,0,0,0,0,0,0,39.91,0,0,0


In [25]:
#整合 function
def hpyer(dataset,ma_index):
    trade=[[0 for i in range(11)]for j in range(len(dataset))]
    trade[0][0]='date'
    trade[0][1]='buy'
    trade[0][2]='hold'
    trade[0][3]='sell'
    trade[0][4]='number'
    trade[0][5]='value'
    trade[0][6]='period_return'
    trade[0][7]='closeprice'
    trade[0][8]='10ma'
    trade[0][9]='holding_period'
    trade[0][10]='annualized_return'
    
    for i in range(1,len(trade)):
        trade[i][0]=dataset[i][0]
        trade[i][7]=float(dataset[i][7])
        
    #ma_index
    for i in range(ma_index,len(trade)):
        temp=0
        for j in range(0,ma_index):
            temp+=trade[i-j][7]
        trade[i][8]=temp/ma_index
        
    for i in range(ma_index,len(trade)):
        if(trade[i][7]>trade[i][8])and(trade[i-1][2]==0):
            trade[i][1]=1
            trade[i][2]=1
            trade[i][3]=0
        elif(trade[i][7]<trade[i][8])and(trade[i-1][2]==1):
            trade[i][1]=0
            trade[i][2]=0
            trade[i][3]=1
        elif(trade[i-1][2]>0):
            trade[i][1]=0
            trade[i][2]=1
            trade[i][3]=0
    for i in range(ma_index,len(trade)):
        if(trade[i][1]==1)and(trade[i][2]==1)and(trade[i][3]==0):
            trade[i][4]=initial/trade[i][7]
            trade[i][5]=trade[i][4]*trade[i][7]
            trade[i][6]=round((trade[i][4]*trade[i][7]-initial)/initial,4)
        elif(trade[i][1]==0)and(trade[i][2]==0)and(trade[i][3]==1):
            trade[i][4]=0
            trade[i][5]=trade[i-1][4]*trade[i][7]
            trade[i][6]=round((trade[i-1][4]*trade[i][7]-initial)/initial,4)
        elif(trade[i][1]==0)and(trade[i][2]==1)and(trade[i][3]==0):
            trade[i][4]=trade[i-1][4]
            trade[i][5]=trade[i][4]*trade[i][7]
            trade[i][6]=round((trade[i][4]*trade[i][7]-initial)/initial,4)
    import datetime
    from datetime import date
    for i in range(1,len(trade)):
        trade[i][0]=datetime.datetime.strptime(trade[i][0],'%Y-%m-%d')
    
    start=0
    end=0
    for i in range(1,len(trade)):
        if(trade[i][1]==1):
            trade[i][9]=1
            start=trade[i][0]
        elif((trade[i][2]==1)or(trade[i][3]==1)and(trade[i][1]!=1)):
            end=trade[i][0]
            trade[i][9]=(end-start).days+1
    for i in range(1,len(trade)):
        if(trade[i][9]!=0):
            trade[i][10]=(1+trade[i][6])**(365/trade[i][9])-1
    annual_return=1
    trading_counts=0
    for i in range(1,len(trade)):
        if(trade[i][3]==1):
            annual_return*=(1+trade[i][6])
            trading_counts+=1
    final_return=annual_return**(1/trading_counts)-1
    return final_return

In [24]:
pd.DataFrame(hpyer(dataset,10)).head()

,0,1,2,3,4,5,6,7,8,9,10
0,date,buy,hold,sell,number,value,period_return,closeprice,10ma,holding_period,annualized_return
1,2003-07-01 00:00:00,0,0,0,0,0,0,38.05,0,0,0
2,2003-07-02 00:00:00,0,0,0,0,0,0,38.69,0,0,0
3,2003-07-03 00:00:00,0,0,0,0,0,0,39,0,0,0
4,2003-07-04 00:00:00,0,0,0,0,0,0,39.26,0,0,0


In [26]:
for i in range(2,10):
    print("This is annual_return of ma"+str(i)+" is "+str(hpyer(dataset,i)))

This is annual_return of ma2 is 0.0006106951900699542
This is annual_return of ma3 is -0.00016588926514460312
This is annual_return of ma4 is -0.00016331547396630786
This is annual_return of ma5 is -0.0005151006523820278
This is annual_return of ma6 is -0.000818800676472331
This is annual_return of ma7 is -0.0003124742224202004
This is annual_return of ma8 is -0.0008119553272980529
This is annual_return of ma9 is -1.4384400451761259e-05
